<a href="https://colab.research.google.com/github/saeidsaadatigero/Computer-Vision/blob/main/vision_computer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow opencv-python

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# بارگذاری مدل YOLO
def load_yolo():
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return net, output_layers

# تشخیص اجزا
def detect_objects(img, net, output_layers):
    height, width, _ = img.shape

    # ایجاد blob
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    boxes = []
    confidences = []
    class_ids = []

    # پردازش خروجی‌ها
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # آستانه اطمینان
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # مستطیل
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # حذف جعبه‌های تکراری
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    return boxes, confidences, class_ids, indexes

# بارگذاری نام کلاس‌ها
def load_classes():
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    return classes

# نمایش نتایج
def draw_labels(boxes, confidences, class_ids, indexes, classes, img):
    for i in indexes:
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = str(round(confidences[i], 2))
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(img, label + " " + confidence, (x, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

# اجرای برنامه
if __name__ == "__main__":
    net, output_layers = load_yolo()
    classes = load_classes()

    # بارگذاری تصویر
    img = cv2.imread("image.jpg")  # نام تصویر را تغییر دهید
    boxes, confidences, class_ids, indexes = detect_objects(img, net, output_layers)
    draw_labels(boxes, confidences, class_ids, indexes, classes, img)

    # نمایش تصویر
    cv2.imshow("Image", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

error: OpenCV(4.10.0) /io/opencv/modules/dnn/src/darknet/darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'readNetFromDarknet'
